In [ ]:
import numpy as np
import pandas as pd

import bebi103
import cmdstanpy
import arviz as az

import bokeh.plotting
import bokeh.io
from bokeh.io import output_file, save
bokeh.io.output_notebook()

$\dot{A}=gA$ 

$\dot{g}=-\alpha(g-\beta \frac{A_{end}-A}{A_{end}})$ if $A<A_{cut}$

Combine Dev and Reg with differnt initial conditions

In [2]:
rng = np.random.default_rng()

from scipy.integrate import solve_ivp

def A_theor(t, A_0, g_0, alpha, beta, A_end,A_cut):
    t48 = t - 48

    def ode_system(t, y):
        A, g = y
        dAdt = g*A
        if A < A_cut:
            dgdt = -alpha * (g - beta * (A_end - A_cut) / A_end)
        else:
            dgdt = -alpha * (g - beta * (A_end - A) / A_end)
        return [dAdt, dgdt]

    # Initial conditions
    y0 = [A_0, g_0]

    # Solve the ODE
    solution = solve_ivp(ode_system, [t48.min(), t48.max()], y0, t_eval=t48, method='RK45')
    #print("Solver message:", solution.message)
    return solution.y[0]

def linCrit_ppc(x):
    sigma = 0*np.abs(rng.normal(0, 2.0))
    A_0 = 10**rng.normal(0.3, 0.15)
    g_0 = np.abs(rng.normal(0, 0.1))
    alpha = 10**rng.normal(-0.25, 0.25)
    beta = (alpha/4)*10**rng.normal(0, 0.1)
    A_end = 10**rng.normal(1.0, 0.1)
    A_cut = 2 + 4*rng.beta(5.0,5.0)
    #print(A_0, g_0,alpha, beta, A_end)
    return (rng.normal(A_theor(x, A_0, g_0,alpha, beta, A_end,A_cut), sigma),
            {"A_0": A_0, "g_0": g_0, "alpha": alpha, "beta": beta, "A_end": A_end, "A_cut":A_cut})

In [ ]:
x = np.linspace(48, 144, 30)
for _ in range(10):
    print(linCrit_ppc(x))

In [3]:
x = np.linspace(48, 144, 100)
f_ppc = [linCrit_ppc(x) for _ in range(100)]

p = bokeh.plotting.figure(
    x_axis_label="x",
    y_axis_label="f",
    frame_height=400,
    frame_width=400,
)

for f, params in f_ppc:
    line = p.line(x, f)
    p.add_tools(bokeh.models.HoverTool(
        renderers=[line],
        tooltips=[(key, f"{params[key]}") for key in params.keys()]
    ))

bokeh.io.show(p)

In [ ]:
import os
Sahra_Shivani_path=(r'/media/max_kotz/sahra_shivani_data')
Output_path=os.path.join(Sahra_Shivani_path,'sorted_data')
Curv_Thick_path=os.path.join(Output_path,'for_curv_thick')
hdf5_file_path = os.path.join(Curv_Thick_path, 'scalarGrowthData.h5')

# Aoad the DataFrame from the HDF5 file
df = pd.read_hdf(hdf5_file_path, key='data')

p = bokeh.plotting.figure(
    frame_width=350,
    frame_height=250,
    x_axis_label="time in hpf",
    y_axis_label="Surface Area",
    #x_axis_type="log",
    toolbar_location='above',
)

colors = bokeh.palettes.Category10_3
items = []
for color, (genotype, sub_df) in zip(colors, df.groupby("condition")):
    items.append(
        (
            bokeh.models.LegendItem(
                label=genotype,
                renderers=[
                    p.scatter(
                        sub_df["time in hpf"],
                        sub_df["Surface Area"]*1e-4,
                        color=color,
                    )
                ],
            )
        )
    )

p.add_layout(bokeh.models.Legend(items=items), 'right')
bokeh.io.show(p)

FileNotFoundError: File /media/max_kotz/sahra_shivani_data/sorted_data/sorted_data/for_curv_thick/scalarGrowthData.h5 does not exist

In [26]:
sm = cmdstanpy.CmdStanModel(stan_file='fit_setPoint.stan')

In [15]:
def df_to_data(cat_Dev,cat_Reg):
    filtered_df = df[df['condition'] == cat_Reg]
    t_Reg = filtered_df['time in hpf'].to_numpy()
    A_Reg = filtered_df['Surface Area'].to_numpy()*1e-4
    sorted_indices = np.argsort(t_Reg)
    t_Reg = t_Reg[sorted_indices]
    A_Reg = A_Reg[sorted_indices]

    filtered_df = df[df['condition'] == cat_Dev]
    t_Dev = filtered_df['time in hpf'].to_numpy()
    A_Dev = filtered_df['Surface Area'].to_numpy()*1e-4
    sorted_indices = np.argsort(t_Dev)
    t_Dev = t_Dev[sorted_indices]
    A_Dev = A_Dev[sorted_indices]

    N_ppc = 100
    t_ppc = np.linspace(48, 144, N_ppc)
    
    return dict(t_Reg=t_Reg, A_Reg=A_Reg, N_Reg=len(t_Reg),
                t_Dev=t_Dev, A_Dev=A_Dev, N_Dev=len(t_Dev),
                  N_ppc=N_ppc, t_ppc=t_ppc)

data = df_to_data('Development','Regeneration')

In [16]:
samples = sm.sample(data=data,iter_warmup=3000,iter_sampling=1000,show_console = False )
samples = az.from_cmdstanpy(posterior=samples, posterior_predictive=['A_Dev_ppc','A_Reg_ppc'])
bebi103.stan.check_all_diagnostics(samples)

12:57:10 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

12:58:48 - cmdstanpy - INFO - CmdStan done processing.
12:58:48 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: Exception: ode_rk45:  Failed to integrate to next output time (48) in less than max_num_steps steps (in 'fit_setPoint.stan', line 28, column 8 to column 91) (in 'fit_setPoint.stan', line 103, column 4 to column 88)
	Exception: Exception: ode_rk45:  Failed to integrate to next output time (48) in less than max_num_steps steps (in 'fit_setPoint.stan', line 28, column 8 to column 91) (in 'fit_setPoint.stan', line 103, column 4 to column 88)
	Exception: Exception: ode_rk45: ode parameters and data[3] is inf, but must be finite! (in 'fit_setPoint.stan', line 28, column 8 to column 91) (in 'fit_setPoint.stan', line 103, column 4 to column 88)
	Exception: Exception: ode_rk45: ode parameters and data[3] is inf, but must be finite! (in 'fit_setPoint.stan', line 28, column 8 to column 91) (in 'fit_setPoint.stan', line 103, column 4 to column 88)
	Exception: Exceptio


Effective sample size looks reasonable for all parameters.

Rhat looks reasonable for all parameters.

0 of 4000 (0.0%) iterations ended with a divergence.

0 of 4000 (0.0%) iterations saturated the maximum tree depth of 10.

E-BFMI indicated no pathological behavior.


0

In [17]:
output_file("area_DevREG_ppc_corner_Dev.html")
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=['A_end','A_cut','A_0_Dev','g_0_Dev', 'alpha','beta_'],
        xtick_label_orientation=np.pi/4,
        plot_ecdf=False
    )
)

In [18]:
output_file("area_DevREG_ppc_corner_Reg.html")
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=['A_end','A_cut','A_0_Reg','g_0_Reg', 'alpha','beta_'],
        xtick_label_orientation=np.pi/4,
        plot_ecdf=False
    )
)

In [ ]:
samples

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats

In [19]:
output_file("area_DevREG_ppc_At_Dev.html")
f_ppc = samples.posterior_predictive.A_Dev_ppc.stack(
    {"sample": ("chain", "draw")}
).transpose("sample", "A_Dev_ppc_dim_0")

bokeh.io.show(
    bebi103.viz.predictive_regression(
        f_ppc,
        samples_x=np.array(data["t_ppc"]),
        data=np.dstack((data["t_Dev"], data["A_Dev"])).squeeze(),
        #x_axis_type="log",
        x_axis_label='t',
        y_axis_label='A',
    )
)

In [20]:
output_file("area_DevREG_ppc_At_Reg.html")
f_ppc = samples.posterior_predictive.A_Reg_ppc.stack(
    {"sample": ("chain", "draw")}
).transpose("sample", "A_Reg_ppc_dim_0")

bokeh.io.show(
    bebi103.viz.predictive_regression(
        f_ppc,
        samples_x=np.array(data["t_ppc"]),
        data=np.dstack((data["t_Reg"], data["A_Reg"])).squeeze(),
        #x_axis_type="log",
        x_axis_label='t',
        y_axis_label='A',
    )
)

In [24]:
posterior = samples.posterior

# Dictionary to store mean and confidence intervals for each variable
results ={}

# Loop through each variable in the posterior
for var_name in posterior.data_vars:
    data = posterior[var_name].values.flatten()
    
    results[var_name]=data

print(results)
# Convert the results dictionary to a pandas DataFrame
results_df = pd.DataFrame(results)

# Display the table
print(results_df)
Output_path=os.path.join(Sahra_Shivani_path,'sorted_data')
Curv_Thick_path=os.path.join(Output_path,'for_curv_thick')
results_df.to_csv(os.path.join(Curv_Thick_path,"area_sampled_parameter_results_setPoint.csv"), index=False)

{'A_0_Dev': array([1.72612, 1.96378, 1.69958, ..., 2.41795, 2.57065, 2.40041]), 'A_0_Dev_tilde': array([0.237072, 0.293093, 0.230342, ..., 0.383447, 0.410043, 0.380286]), 'A_0_Reg': array([0.987513, 1.38665 , 1.47249 , ..., 1.17448 , 0.77257 , 1.39131 ]), 'A_0_Reg_tilde': array([-0.00545699,  0.141968  ,  0.168051  , ...,  0.0698449 ,
       -0.112062  ,  0.143425  ]), 'A_cut': array([2.73137, 4.22949, 4.51713, ..., 3.76729, 4.01478, 3.96387]), 'A_cut_tilde': array([-1.4972   ,  0.230505 ,  0.52914  , ..., -0.233768 ,  0.0147825,
       -0.0361382]), 'A_end': array([10.8681, 10.8031, 10.7884, ..., 10.9199, 10.7427, 11.1483]), 'A_end_tilde': array([1.03616, 1.03355, 1.03296, ..., 1.03822, 1.03111, 1.04721]), 'alpha': array([0.333483, 0.35148 , 0.413162, ..., 0.303057, 0.285185, 0.279941]), 'alpha_tilde': array([-0.476926, -0.454099, -0.38388 , ..., -0.518476, -0.544874,
       -0.552934]), 'beta_': array([0.0760402, 0.0724524, 0.0751683, ..., 0.0634942, 0.0750046,
       0.0602441]), 'b

In [ ]:
bebi103.stan.clean_cmdstan()